In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\Govinda sah\\MLOPs\\Mlops_DeepLearning_Project\\research'

In [3]:
os.chdir("..")
print("Current directory:", os.getcwd())

Current directory: c:\Users\Govinda sah\MLOPs\Mlops_DeepLearning_Project


In [4]:
os.getcwd()


'c:\\Users\\Govinda sah\\MLOPs\\Mlops_DeepLearning_Project'

In [5]:
print(os.getcwd())

c:\Users\Govinda sah\MLOPs\Mlops_DeepLearning_Project


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

    

In [7]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import requests
import zipfile
from cnnClassifier import logger
from dataclasses import dataclass

# Setup logger (ensure this is done only once in your main module)

# ✅ Configuration class using dataclass
# @dataclass
# class DataIngestionConfig:
#     source_URL: str         # URL to download ZIP file
#     local_data_file: str    # ZIP file name to save
#     root_dir: str           # Directory to save ZIP file
#     unzip_dir: str          # Directory to extract contents

# ✅ Main Data Ingestion Class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Downloads a ZIP file from a URL and saves it locally.
        Returns:
            str: Path to the downloaded ZIP file.
        """
        url = self.config.source_URL
        rootdir = self.config.root_dir
        filename = self.config.local_data_file
        
        os.makedirs(rootdir, exist_ok=True)
        zip_file_path = os.path.join(rootdir, filename)

        logger.info(f"Starting download from {url}")
        response = requests.get(url)

        if response.status_code == 200:
            with open(zip_file_path, 'wb') as f:
                f.write(response.content)
            logger.info(f"✅ ZIP file saved at: {zip_file_path}")
            return zip_file_path
        else:
            logger.error(f"❌ Download failed with status code: {response.status_code}")
            raise Exception(f"Download failed with status code: {response.status_code}")

    def extract_zip_file(self) -> bool:
        """
        Extracts the downloaded ZIP file into the specified directory.
        Returns:
            bool: True if extraction is successful, else False.
        """
        zip_file_path = os.path.join(self.config.root_dir, self.config.local_data_file)
        unzip_path = self.config.unzip_dir

        if not os.path.exists(zip_file_path):
            logger.error(f"❌ ZIP file not found at: {zip_file_path}")
            return False

        try:
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"✅ ZIP file extracted to: {unzip_path}")
            return True
        except zipfile.BadZipFile:
            logger.error("❌ The file is not a valid ZIP archive.")
            return False


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-14 21:00:41,368: INFO: common: yaml file: C:\Users\Govinda sah\MLOPs\Mlops_DeepLearning_Project\config\config.yaml loaded successfully]
[2025-04-14 21:00:41,378: INFO: common: yaml file: C:\Users\Govinda sah\MLOPs\Mlops_DeepLearning_Project\params.yaml loaded successfully]
[2025-04-14 21:00:41,381: INFO: common: created directory at: artifacts]
[2025-04-14 21:00:41,382: INFO: common: created directory at: artifacts\data_ingestion]
[2025-04-14 21:00:41,384: INFO: 1996508912: Starting download from https://github.com/TechGovinda/Datasets/raw/main/Chicken-fecal-images.zip]
[2025-04-14 21:00:43,540: INFO: 1996508912: ✅ ZIP file saved at: artifacts\data_ingestion\data.zip]
[2025-04-14 21:00:43,942: INFO: 1996508912: ✅ ZIP file extracted to: artifacts\extreacted_repo]
